In [1]:
##The output file says for each cluster and pfam domain, the number of times it appears and the max Score it has
##This output file was not used in the end. Only binary vectors in which the presence or absence (1 or 0) of
##each pfam domain was stated were used for machine learning

C:\Users\Yoo\Desktop\Summer Project\hmmer-3.2.1\trial


In [ ]:
cd C:\Users\Yoo\Desktop\Summer Project\hmmer-3.2.1\trial 

In [2]:
inputFile = open ("everycdspfamdomains.txt","r")
outputFile = open ("trialAllPfamDomains.txt","w")
clusterDictionary={}
currentCluster=""

In [3]:
##This code is kind of tricky. In the input file, the same cluster appears more than once (once for each CDS)
##So we have to know if we are still in the same cluster or if we are in a new one
##The lines that start with ">>" are the lines with the name of a domain
##Exactly three lines after that we find the score of the domain for that cluster
##However, there can be another score just below that one, so we have to turn on the flag "morePossibleScores" till we know for sure there are no more scores
##We know there is no more scores if the next line is empty. That is, the flag is on but "If Line:" returns false
##When the "lookingForTheScoreFlag" is 3, then we get the score as we said that the score is three lines after the domain name
##When checking the other lines with more possible scores, if the score is bigger we keep it, otherwise ignore it
##Each cluster finishes with "//" and we turn the newQueryFlag on
newQueryFlag = True
lookingForTheScoreFlag = 4
morePossibleScores=False
maxScore=0.0
for line in inputFile:
    line=line.strip("\n")
    if morePossibleScores:
        if line:
            #All those empty lines after the the possible scores just have a "\n" that has been remove two lines up
            #Only other lines with a score will be true then
            line=" ".join(line.split())
            line=line.split()
            if float(line[2]) > maxScore:
                maxScore=float(line[2])
        else:
            if clusterDictionary[currentCluster][domainName][1]<maxScore:
                clusterDictionary[currentCluster][domainName][1]=maxScore
            maxScore=0.0
            morePossibleScores=False
            lookingForTheScoreFlag=4
    elif newQueryFlag:
        if line[:6]=="Query:":
            line=" ".join(line.split())
            line=line.split("|")
            currentCluster=line[0][7:]
            if currentCluster not in clusterDictionary.keys():
                clusterDictionary[currentCluster]={}
            newQueryFlag = False
    else:
        if lookingForTheScoreFlag==4:
            if line[:2]=="//":
                newQueryFlag=True
            elif line[:3]==">> ":
                domainName=line[3:].split(" ")[0]
                if domainName not in clusterDictionary[currentCluster].keys():
                    clusterDictionary[currentCluster][domainName]=[1,0.0]
                else:
                    clusterDictionary[currentCluster][domainName][0] += 1
                lookingForTheScoreFlag=0
        else:
            lookingForTheScoreFlag +=1
            if lookingForTheScoreFlag==3:
                line=" ".join(line.split())
                line=line.split()
                maxScore=float(line[2])
                morePossibleScores=True

In [4]:
print (clusterDictionary)

{'BGC0000001': {'PCMT': [1, 89.4], 'Methyltransf_31': [1, 38.2], 'Methyltransf_25': [1, 33.2], 'Methyltransf_3': [1, 19.9], 'Methyltransf_23': [1, 20.1], 'Methyltransf_11': [1, 18.6], 'Fibrillarin': [1, 15.4], 'CMAS': [1, 15.2], 'ACP_syn_III_C': [2, 29.7], 'Methyltransf_12': [1, 16.0], 'GCD14': [1, 14.9], 'MTS': [1, 13.3], 'Methyltransf_24': [1, 14.0], 'Methyltransf_32': [1, 13.2], 'Methyltransf_18': [1, 13.0], 'RrnaAD': [1, 11.4], 'Ubie_methyltran': [1, 11.4], 'Methyltransf_16': [1, 11.3], 'BTAD': [2, 137.1], 'Trans_reg_C': [2, 36.9], 'TPR_10': [3, 15.5], 'TPR_14': [2, 17.3], 'p450': [2, 95.3], 'GerE': [1, 55.5], 'AAA_16': [2, 27.0], 'TPR_2': [2, 21.9], 'TPR_12': [2, 21.4], 'TPR_19': [2, 20.0], 'AAA_22': [2, 18.4], 'Sigma70_r4_2': [1, 14.7], 'HTH_10': [2, 11.7], 'AAA_30': [2, 12.3], 'TPR_7': [1, 20.4], 'TPR_16': [1, 20.3], 'TPR_1': [1, 19.9], 'TPR_6': [1, 9.2], 'TPR_8': [1, 12.6], 'RHS_repeat': [1, 26.5], 'DUF4595': [1, 11.7], 'DUF2850': [1, 9.6], 'TruB_C': [1, 6.7], 'ACP_syn_III': [4

In [5]:
##Writes to a file the whole cluster dictionary. Key is the cluster name and insideKey is the name of a domain
##For each domain we have [X, Y] where X is the number of times and Y is the max score
for key in clusterDictionary.keys():
    line=key+": "
    for insideKey in sorted(clusterDictionary[key].keys()):
        line+= insideKey + ": " + str(clusterDictionary[key][insideKey]) +"; "
    outputFile.write(line+"\n")

In [6]:
inputFile.close()
outputFile.close()